In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
import tensorflow as tf
from utils import download, extract, get_cifar10_batches, get_cifar10_test_batch

In [2]:
cifar_10_download = Path("data/cifar-10-python.tar.gz")
if not cifar_10_download.exists():
    download("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", cifar_10_download)

In [3]:
extract_loc = Path("data/cifar-10-batches-py")
if not extract_loc.exists():
    extract(cifar_10_download)

In [4]:
batches = get_cifar10_batches()
test_batch = get_cifar10_test_batch()

In [5]:
n_epochs = 10
batch_size = 64
n = 32
n_channels = 3
n_samples = batches.shape[0]

In [6]:
inp = tf.placeholder(tf.float32, [batch_size, n, n, n_channels])
category = tf.placeholder(tf.int32, [batch_size, 1])
conv1 = tf.layers.conv2d(
    inputs=inp,
    filters=32,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)

pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

# Convolutional Layer #2 and Pooling Layer #2
conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

# Dense Layer
pool2_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dropout = tf.layers.dropout(inputs=dense, rate=0.4)

# Logits Layer
logits = tf.layers.dense(inputs=dropout, units=10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=category, logits=logits)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(
    loss=loss,
    global_step=tf.train.get_global_step())

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for ite in range(n_samples // batch_size):
        batch = batches.sample(n=batch_size)
        x, y = batch[1], batch[0]
        l, _ = sess.run([loss, train_op], feed_dict={inp: np.stack(list(x)).reshape([batch_size, n, n, n_channels]), category: np.stack(list(y)).reshape([batch_size, 1])})
        this_test_batch = test_batch.sample(n=batch_size)
        test_x, test_y = this_test_batch[1], this_test_batch[0]
        t_l = sess.run([loss], feed_dict={inp: np.stack(list(test_x)).reshape([batch_size, n, n, n_channels]), category: np.stack(list(test_y)).reshape([batch_size, 1])})
        print("iter: {}, train loss: {:.2f}, test loss: {}".format(ite, l, t_l))

iter: 0, train loss: 61.58, test loss: [941.1078]
iter: 1, train loss: 820.25, test loss: [30.243958]
iter: 2, train loss: 43.11, test loss: [58.493683]
iter: 3, train loss: 55.17, test loss: [3.3320942]
iter: 4, train loss: 2.86, test loss: [2.842989]
iter: 5, train loss: 2.35, test loss: [2.3068764]
iter: 6, train loss: 2.31, test loss: [2.337867]
iter: 7, train loss: 2.40, test loss: [2.7994213]
iter: 8, train loss: 2.77, test loss: [2.4857194]
iter: 9, train loss: 4.79, test loss: [2.3434677]
iter: 10, train loss: 2.29, test loss: [2.2797074]
iter: 11, train loss: 2.34, test loss: [2.3240404]
iter: 12, train loss: 2.29, test loss: [2.3757832]
iter: 13, train loss: 2.83, test loss: [2.3269143]
iter: 14, train loss: 2.30, test loss: [2.2861545]
iter: 15, train loss: 2.39, test loss: [2.3739185]
iter: 16, train loss: 2.30, test loss: [2.4947348]
iter: 17, train loss: 2.57, test loss: [2.307457]
iter: 18, train loss: 2.98, test loss: [2.3006496]
iter: 19, train loss: 2.34, test loss: [